In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 61.8 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [2]:
import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
print("⏳ Đang tải dữ liệu CAFA 6...")
!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json
⏳ Đang tải dữ liệu CAFA 6...
  0% 0.00/91.3M [00:00<?, ?B/s]
100% 91.3M/91.3M [00:00<00:00, 1.76GB/s]
✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import gc
import os
import sys
from tqdm.auto import tqdm

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    'EMBEDDINGS': {
        'ankh': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_embeddings.npy',
        },
        'esm': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_test_FINAL_embeddings.npy',
        },
        'prot_t5': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_test_FINAL_embeddings.npy',
        },
        'taxonomy': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings_2/Taxonomy/train_taxonomy_indices.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings_2/Taxonomy/test_taxonomy_indices.npy',
        }
        # 'protBERT': {
        #     'train': '/content/drive/MyDrive/CAFA6_Embeddings/protBERT_Aligned/protBERT_train_FINAL_embeddings.npy',
        #     'test':  '/content/drive/MyDrive/CAFA6_Embeddings/protBERT_Aligned/protBERT_test_FINAL_embeddings.npy',
        # },
    },

    'TRAIN_ID_PATH': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_ids.npy',
    'TEST_ID_PATH':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_ids.npy',

    'SAVE_DIR': '/content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run',
    'TRAIN_TERMS': '/content/cafa6_data/Train/train_terms.tsv',
    'IA_FILE': '/content/cafa6_data/IA.tsv',
    'TEST_FASTA': '/content/cafa6_data/Test/testsuperset.fasta',

    # Model Params
    'TOP_K_LABELS': 10000, # Giữ mức này để model học tốt
    # 'MIN_FREQ': 3,

    # Input sẽ là 768 (Ankh) + 1280 (ESM) = 2048
    'ENCODER_LAYERS': [2048, 1024, 512],
    'HEAD_HIDDEN_DIM': 512,

    'TOP_K_SPECIES': 100,         # Giữ 100 loài phổ biến nhất
    'TAXONOMY_EMBEDDING_DIM': 8,

    'DROPOUT_RATE': 0.4, # Tăng dropout vì input lớn
    'EPOCHS': 50,
    'BATCH_SIZE': 128,
    'LEARNING_RATE': 2e-4, # Giảm LR một chút cho ổn định
    'LABEL_SMOOTHING': 0.1,

    # Inference Params (Chuẩn 0.27 điểm)
    'MIN_CONFIDENCE': 0.01,
    'MAX_PREDS_PER_PROTEIN': 20,

    'SEED': 42,
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu'
}

os.makedirs(CONFIG['SAVE_DIR'], exist_ok=True)
print(f"🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: {CONFIG['DEVICE']}")

torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

def preprocess_taxonomy_shift(train_path, test_path, top_k=100):
    """
    Hàm này cực kỳ quan trọng:
    1. Đếm tần suất loài trong tập Train.
    2. Map Top K loài phổ biến nhất sang index [1, 2, ..., K].
    3. Tất cả loài còn lại (Unseen, Rare) sang index [K+1].
    4. Index 0 để dành làm Padding.
    """
    print(f"\n🦖 Preprocessing Taxonomy (SHIFT MODE): Keeping Top {top_k} species...")

    # 1. Load dữ liệu gốc (Đã được map 0, 1, 2...)
    train_tax = np.load(train_path)
    test_tax = np.load(test_path)

    # 2. Đếm tần suất
    unique, counts = np.unique(train_tax, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_ids = unique[sorted_indices]

    # Lấy Top K ID
    top_ids = sorted_ids[:top_k]

    print(f"   Top 1 species ID (Original): {top_ids[0]} (Count: {counts[sorted_indices][0]})")

    # 3. Tạo Mapping Table
    max_id = max(train_tax.max(), test_tax.max())
    lookup_table = np.zeros(max_id + 1, dtype=np.int64)

    # --- QUY HOẠCH SỐ NHÀ ---
    # Index 0: Padding (Trống)
    # Index 1..K: Top Species
    # Index K+1: Other

    other_idx = top_k + 1
    lookup_table[:] = other_idx # Mặc định gán tất cả là Other

    # Gán index mới cho Top K (Cộng 1 để tránh số 0)
    for new_idx, original_id in enumerate(top_ids):
        # original_id=0 (Người) -> new_idx=0 -> lookup=1. (Người chuyển sang số 1)
        lookup_table[original_id] = new_idx + 1

    # 4. Map lại dữ liệu
    new_train_tax = lookup_table[train_tax]
    new_test_tax = lookup_table[test_tax]

    # Check lại số 0
    # lookup_table[0] là cái gì?
    # Nếu Top 1 là 0, thì lookup_table[0] = 1.
    # Vậy nên input sẽ KHÔNG CÒN SỐ 0 NỮA -> An toàn cho Padding.

    print(f"   ✅ Remapped Taxa. New shape: Train {new_train_tax.shape}, Test {new_test_tax.shape}")
    print(f"   Example: Original {train_tax[:5]} -> New {new_train_tax[:5]}")

    # Tổng số class embedding = (Other) + 1 (vì index max là other_idx) + 1 (Padding 0)
    # Thực ra chỉ cần khai báo num_embeddings = other_idx + 1 là đủ (chứa từ 0 đến other_idx)
    return new_train_tax, new_test_tax, other_idx + 1

# ============================================================================
# 1. MEMORY-SAFE DATASET (CHÌA KHÓA ĐỂ KHÔNG TRÀN RAM)
# ============================================================================
class MultiSourceDataset(Dataset):
    def __init__(self, embedding_paths_dict, y_tensor=None, indices=None, override_data=None):
        self.mmaps = {}
        self.override_data = override_data if override_data else {}
        self.keys = list(embedding_paths_dict.keys())

        # Load mmap (chỉ những cái không override)
        for name, path in embedding_paths_dict.items():
            if name not in self.override_data:
                self.mmaps[name] = np.load(path, mmap_mode='r')

        # Base length
        first_key = self.keys[0]
        if first_key in self.override_data:
            self.total_len = len(self.override_data[first_key])
        else:
            self.total_len = len(self.mmaps[first_key])

        self.indices = indices if indices is not None else np.arange(self.total_len)
        self.y = y_tensor

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        inputs = []
        for key in self.keys:
            if key in self.override_data:
                # Dữ liệu từ RAM (Taxonomy đã xử lý)
                data = self.override_data[key][real_idx]
                if 'taxonomy' in key:
                    vec = torch.tensor(data, dtype=torch.long)
                else:
                    vec = torch.tensor(data, dtype=torch.float)
            else:
                # Dữ liệu từ Disk (Ankh, ESM)
                data = self.mmaps[key][real_idx].copy()
                if 'taxonomy' in key:
                    vec = torch.tensor(data, dtype=torch.long)
                else:
                    vec = torch.from_numpy(data).float()
            inputs.append(vec)

        if self.y is not None:
            return inputs, self.y[real_idx]
        return (inputs,)

# ============================================================================
# 2. LOAD DATA & PROCESS LABELS
# ============================================================================

train_tax_path = CONFIG['EMBEDDINGS']['taxonomy']['train']
test_tax_path = CONFIG['EMBEDDINGS']['taxonomy']['test']

new_train_tax, new_test_tax, num_taxa_classes = preprocess_taxonomy_shift(
    train_tax_path, test_tax_path, top_k=CONFIG['TOP_K_SPECIES']
)

# Cập nhật Config
CONFIG['NUM_TAXA'] = num_taxa_classes
print(f"👉 FINAL NUM_TAXA for Embedding Layer: {CONFIG['NUM_TAXA']} (Includes 0=Padding, {CONFIG['NUM_TAXA']-1}=Other)")

# Tạo dictionary để override
train_override = {'taxonomy': new_train_tax}
test_override = {'taxonomy': new_test_tax}

print("\n[1/5] Checking Files...")
train_ids = np.load(CONFIG['TRAIN_ID_PATH'])
print(f"   ✓ Train IDs: {len(train_ids)}")

print("\n[2/5] Processing Labels (IA Strategy)...")
# 1. Load Terms & IA
df_terms = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', header=0, names=['EntryID', 'term', 'aspect'])
df_ia = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'])
ia_dict = dict(zip(df_ia['term'], df_ia['ia']))

# 2. Tính Score
term_counts = df_terms['term'].value_counts().reset_index()
term_counts.columns = ['term', 'freq']

# print(f"   Original terms count: {len(term_counts)}")
# term_counts = term_counts[term_counts['freq'] >= CONFIG['MIN_FREQ']]
# print(f"   -> After filtering (Freq >= {CONFIG['MIN_FREQ']}): {len(term_counts)} terms")

term_counts['ia'] = term_counts['term'].map(ia_dict).fillna(0.0)
term_counts['score'] = term_counts['freq'] * term_counts['ia']

# 3. Chọn Top K
top_terms_df = term_counts.sort_values(by='score', ascending=False).head(CONFIG['TOP_K_LABELS'])
top_terms = top_terms_df['term'].tolist()
print(f"   ✓ Selected {len(top_terms)} terms.")

# 4. Map ID -> Terms
df_filtered = df_terms[df_terms['term'].isin(top_terms)]
id_to_terms = df_filtered.groupby('EntryID')['term'].apply(list).to_dict()

# Dọn dẹp
del df_terms, df_filtered, df_ia, term_counts, top_terms_df
gc.collect()

# ============================================================================
# 3. PREPARE LABELS (RAM OPTIMIZED)
# ============================================================================
print("\n[3/5] Preparing Labels (Sparse Mode)...")

# 1. Sparse MLB
mlb = MultiLabelBinarizer(classes=top_terms, sparse_output=True)
mlb.fit([top_terms])

# 2. Transform -> Sparse Matrix
y_labels_list = [id_to_terms.get(pid, []) for pid in train_ids]
y_train_sparse = mlb.transform(y_labels_list)
del y_labels_list, train_ids # Xóa ID list không dùng nữa
gc.collect()

# 3. Weights (Optional)
# Nếu dùng IA weight thì giữ, nếu muốn model tự học thì comment dòng này và bỏ pos_weight trong Loss
weights_list = [ia_dict.get(t, 0.0) for t in mlb.classes_]
pos_weight_tensor = torch.tensor(weights_list, dtype=torch.float32).to(CONFIG['DEVICE'])

# 4. Convert to Dense Float32 & Label Smoothing
print("   ⏳ Converting Labels to Tensor...")
# Convert từng phần nhỏ hoặc convert hết nếu RAM > 12GB (với 80k row x 10k col float32 ~ 3.2GB -> Ổn)
y_train_binary = y_train_sparse.astype(np.float32).toarray()
y_train_tensor = torch.from_numpy(y_train_binary)

if CONFIG['LABEL_SMOOTHING'] > 0:
    y_train_tensor.mul_(1 - CONFIG['LABEL_SMOOTHING']).add_(CONFIG['LABEL_SMOOTHING'] / len(top_terms))

del y_train_sparse, y_train_binary
gc.collect()

# ============================================================================
# 4. CREATE DATA LOADERS
# ============================================================================
print("\n[4/5] Creating DataLoaders...")

train_paths = {k: v['train'] for k, v in CONFIG['EMBEDDINGS'].items()}

indices = np.arange(len(y_train_tensor))
train_idx, val_idx = train_test_split(indices, test_size=0.15, random_state=CONFIG['SEED'])

train_dataset = MultiSourceDataset(train_paths, y_train_tensor, indices=train_idx, override_data=train_override)
val_dataset   = MultiSourceDataset(train_paths, y_train_tensor, indices=val_idx, override_data=train_override)

# --- AUTO DETECT INPUT DIMS ---
print("   ⏳ Measuring input shapes from dataset...")
sample_inputs, _ = train_dataset[0]
INPUT_DIMS_LIST = []
for x in sample_inputs:
    if x.ndim > 0:
        INPUT_DIMS_LIST.append(x.shape[0])
    else:
        INPUT_DIMS_LIST.append(1)
print(f"   ✓ Detected Input Dims: {INPUT_DIMS_LIST}")

train_loader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

# ============================================================================
# 5. MODEL & TRAINING
# ============================================================================
print("\n[5/5] Building Model...")

class EncoderBlock(nn.Module):
    def __init__(self, in_dim, layers_config, dropout):
        super().__init__()
        layers = []

        # 1. LAYER NORM ĐẦU VÀO (BẮT BUỘC)
        # Để cân bằng 'âm lượng' giữa ESM (hét to) và Ankh (nói nhỏ)
        layers.append(nn.LayerNorm(in_dim))

        prev_dim = in_dim
        for i, dim in enumerate(layers_config):
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim), # Ổn định training
                nn.GELU(),           # Hiện đại hơn ReLU
                nn.Dropout(dropout)
            ])
            prev_dim = dim

        self.net = nn.Sequential(*layers)
        self.out_dim = prev_dim

    def forward(self, x):
        return self.net(x)

# Module lọc nhiễu (Attention)
class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block để lọc nhiễu sau khi gộp"""
    def __init__(self, in_dim, reduction=4):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, in_dim // reduction, bias=False),
            nn.ReLU(),
            nn.Linear(in_dim // reduction, in_dim, bias=False),
            nn.Sigmoid() # Tạo ra mask từ 0 đến 1
        )

    def forward(self, x):
        # x shape: [Batch, Dim]
        # Attention weight: [Batch, Dim]
        w = self.fc(x)
        # Nhân trọng số vào x: Cái nào quan trọng thì giữ, rác thì nhân với 0
        return x * w

class MultiModalNet(nn.Module):
    def __init__(self, input_dims_list, encoder_layers, dropout, num_classes, head_hidden_dim, num_taxa, taxonomy_embedding_dim):
        super().__init__()

        self.encoders = nn.ModuleList()
        self.fusion_input_dim = 0
        self.num_taxa = num_taxa

        print("\n🏗️ Building Advanced Architecture:")

        num_prot_embs = len(input_dims_list)
        if num_taxa > 0:
            num_prot_embs -= 1

        # 1. Xây dựng các nhánh Encoder
        for i in range(num_prot_embs):
            in_dim = input_dims_list[i]
            print(f"   ➤ Branch {i+1}: Input {in_dim} -> Output {encoder_layers[-1]}")
            enc = EncoderBlock(in_dim, encoder_layers, dropout)
            self.encoders.append(enc)
            self.fusion_input_dim += enc.out_dim

        if self.num_taxa > 0:
            print(f"   ➤ Taxonomy Branch: Embedding({num_taxa} -> {taxonomy_embedding_dim}) -> Output 64")

            self.tax_encoder = nn.Sequential(
                # Layer 1: Học vector đặc trưng từ index loài
                # padding_idx=0: Loài "Unknown" (index 0) sẽ luôn có vector toàn số 0
                nn.Embedding(num_embeddings=num_taxa, embedding_dim=taxonomy_embedding_dim, padding_idx=0),

                # Layer 2: Xử lý vector đó
                nn.Linear(taxonomy_embedding_dim, taxonomy_embedding_dim),
                nn.BatchNorm1d(taxonomy_embedding_dim),
                nn.SiLU(),
                nn.Dropout(dropout)
            )
            self.fusion_input_dim += taxonomy_embedding_dim # Cộng thêm 64 chiều vào tổng

        print(f"   ➤ Total Fusion Dim: {self.fusion_input_dim}")
        print(f"   ➤ Head Hidden Dim:  {head_hidden_dim}")

        # 2. SE-Block (Bộ lọc thông minh)
        self.attention_filter = SEBlock(self.fusion_input_dim)

        # 3. Layer tổng hợp cuối cùng
        self.head = nn.Sequential(
            nn.BatchNorm1d(self.fusion_input_dim),
            nn.Linear(self.fusion_input_dim, head_hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),

            # Thêm một lớp nữa để tăng khả năng học
            nn.Linear(head_hidden_dim, head_hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),

            nn.Linear(head_hidden_dim, num_classes)
        )

    def forward(self, inputs_list):
        features = []

        # 1. Tách Input
        if self.num_taxa > 0:
            prot_inputs = inputs_list[:-1] # List các tensor float (Ankh, ESM...)
            tax_input = inputs_list[-1]    # Tensor int (Taxonomy Indices)
        else:
            prot_inputs = inputs_list
            tax_input = None

        # 2. Xử lý Protein Encoders
        for i, encoder in enumerate(self.encoders):
            feat = encoder(prot_inputs[i])
            features.append(feat)

        # 3. Xử lý Taxonomy (Nếu có)
        if self.num_taxa > 0:
            # Đảm bảo tax_input là LongTensor (số nguyên) và đúng chiều
            if tax_input.dtype != torch.long:
                tax_input = tax_input.long()

            # Nếu shape là [Batch, 1] thì squeeze thành [Batch]
            if tax_input.dim() == 2:
                tax_input = tax_input.squeeze(1)

            tax_feat = self.tax_encoder(tax_input)
            features.append(tax_feat)

        # 4. Gộp (Concatenate)
        combined = torch.cat(features, dim=1)

        # 5. Lọc nhiễu & Phân loại
        refined = self.attention_filter(combined)
        return self.head(refined)

model = MultiModalNet(
    input_dims_list = INPUT_DIMS_LIST,
    encoder_layers  = CONFIG['ENCODER_LAYERS'],
    dropout         = CONFIG['DROPOUT_RATE'],
    num_classes     = len(top_terms),
    head_hidden_dim = CONFIG['HEAD_HIDDEN_DIM'],
    num_taxa        = CONFIG['NUM_TAXA'],             # 8453
    taxonomy_embedding_dim = CONFIG['TAXONOMY_EMBEDDING_DIM'] # 64
).to(CONFIG['DEVICE'])

# Loss & Optimizer
# Có thể thử bỏ pos_weight nếu thấy loss dao động quá mạnh
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['LEARNING_RATE'], weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

print(f"\n🚀 START TRAINING ({CONFIG['EPOCHS']} Epochs)...")
best_val_loss = float('inf')
best_model_path = f"{CONFIG['SAVE_DIR']}/prott5_esm2_ankh_best.pth"

for epoch in range(CONFIG['EPOCHS']):
    model.train()
    train_loss = 0
    for X_b, y_b in train_loader:
        X_b, y_b = [x.to(CONFIG['DEVICE']) for x in X_b], y_b.to(CONFIG['DEVICE'])
        optimizer.zero_grad()
        logits = model(X_b)
        loss = criterion(logits, y_b)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item()

    avg_train = train_loss / len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_b, y_b in val_loader:
            X_b, y_b = [x.to(CONFIG['DEVICE']) for x in X_b], y_b.to(CONFIG['DEVICE'])
            logits = model(X_b)
            loss = criterion(logits, y_b)
            val_loss += loss.item()

    avg_val = val_loss / len(val_loader)
    scheduler.step(avg_val)

    print(f"Epoch {epoch+1:02d} | Train: {avg_train:.4f} | Val: {avg_val:.4f} | LR: {optimizer.param_groups[0]['lr']:.1e}")

    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), best_model_path)
        print("  ⭐ New Best Model!")

# ============================================================================
# 6. INFERENCE (STREAMING)
# ============================================================================
print("\n🔮 PREDICTING...")

# Dọn dẹp Training
del train_loader, val_loader, train_dataset, val_dataset, y_train_tensor, optimizer
gc.collect()
torch.cuda.empty_cache()

model.load_state_dict(torch.load(best_model_path))
model.eval()

# Load Test IDs & Paths
test_paths = {k: v['test'] for k, v in CONFIG['EMBEDDINGS'].items()}
test_ids = np.load(CONFIG['TEST_ID_PATH'])

# Test Dataset (Dual Memmap)
test_dataset = MultiSourceDataset(test_paths, override_data=test_override)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['BATCH_SIZE']*2, shuffle=False, num_workers=2)

submission_path = f"{CONFIG['SAVE_DIR']}/submission.tsv"
n_predictions = 0

with open(submission_path, 'w') as f:
    current_idx = 0
    with torch.no_grad():
        for (X_b,) in tqdm(test_loader, desc="Inference"):
            X_b = [x.to(CONFIG['DEVICE']) for x in X_b]
            logits = model(X_b)
            probs_batch = torch.sigmoid(logits).cpu().numpy()

            ids_batch = test_ids[current_idx : current_idx + len(probs_batch)]
            current_idx += len(probs_batch)

            for i, pid in enumerate(ids_batch):
                probs = probs_batch[i]

                # TOP-K CỨNG (Chiến thuật 0.27 điểm)
                top_k = CONFIG['MAX_PREDS_PER_PROTEIN']
                ind = np.argpartition(probs, -top_k)[-top_k:]
                ind = ind[np.argsort(probs[ind])][::-1]

                for idx in ind:
                    score = probs[idx]
                    if score > CONFIG['MIN_CONFIDENCE']:
                        f.write(f"{pid}\t{top_terms[idx]}\t{score:.3f}\n")
                        n_predictions += 1

            del probs_batch, X_b, logits

print(f"\n✅ DONE! File: {submission_path}")
print(f"\n✅ DONE! Predictions: {n_predictions:,}")

🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: cuda

🦖 Preprocessing Taxonomy (SHIFT MODE): Keeping Top 100 species...
   Top 1 species ID (Original): 0 (Count: 17162)
   ✅ Remapped Taxa. New shape: Train (82404,), Test (224309,)
   Example: Original [0 0 0 0 0] -> New [1 1 1 1 1]
👉 FINAL NUM_TAXA for Embedding Layer: 102 (Includes 0=Padding, 101=Other)

[1/5] Checking Files...
   ✓ Train IDs: 82404

[2/5] Processing Labels (IA Strategy)...
   ✓ Selected 10000 terms.

[3/5] Preparing Labels (Sparse Mode)...
   ⏳ Converting Labels to Tensor...

[4/5] Creating DataLoaders...
   ⏳ Measuring input shapes from dataset...
   ✓ Detected Input Dims: [768, 1280, 1024, 1]

[5/5] Building Model...

🏗️ Building Advanced Architecture:
   ➤ Branch 1: Input 768 -> Output 512
   ➤ Branch 2: Input 1280 -> Output 512
   ➤ Branch 3: Input 1024 -> Output 512
   ➤ Taxonomy Branch: Embedding(102 -> 8) -> Output 64
   ➤ Total Fusion Dim: 1544
   ➤ Head Hidden Dim:  512

🚀 START TRAINING (50 Epochs)...
Epoch 01

Inference:   0%|          | 0/877 [00:00<?, ?it/s]


✅ DONE! File: /content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run/submission.tsv

✅ DONE! Predictions: 4,285,684


In [5]:
# Chạy lệnh này trong một cell mới
!kaggle competitions submit \
    -c cafa-6-protein-function-prediction \
    -f /content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run/submission.tsv \
    -m "esm2 ankh prot t5 taxon"

100% 98.4M/98.4M [00:02<00:00, 46.5MB/s]
Successfully submitted to CAFA 6 Protein Function Prediction

In [6]:
!kaggle competitions submissions -c cafa-6-protein-function-prediction

fileName        date                        description                                                                              status                     publicScore  privateScore  
--------------  --------------------------  ---------------------------------------------------------------------------------------  -------------------------  -----------  ------------  
submission.tsv  2025-12-16 00:44:55.140000  esm2 ankh prot t5 taxon                                                                  SubmissionStatus.COMPLETE  0.252                      
submission.tsv  2025-12-15 18:00:19.587000  esm2 ankh prot t5 taxon                                                                  SubmissionStatus.COMPLETE  0.258                      
submission.tsv  2025-12-15 16:17:45.733000  esm2 ankh prot t5 chunk_aspect                                                           SubmissionStatus.COMPLETE  0.251                      
submission.tsv  2025-12-14 09:32:43.093000  merge DL 0.8 ML 